In [1]:
import datetime
from queue import *
import pandas as pd
import datetime as dt
import pickle
import time
#from geopy.distance import distance as geo_dist
import scipy.stats
from bisect import bisect_right
import getpass
import pyspark
import numpy as np
from pyspark.sql import SparkSession
import math
from pyspark.sql.types import IntegerType
import pyspark.sql.functions as f
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType
from datetime import datetime, timedelta
from scipy import stats
from heapq import *
spark = SparkSession \
    .builder \
    .master("yarn") \
    .appName('sbb2-{0}'.format(getpass.getuser())) \
    .config('spark.executor.memory', '4g') \
    .config('spark.executor.instances', '5') \
    .config('spark.port.maxRetries', '100') \
    .getOrCreate()
spark

**In this notebook, we will build small toy example to discuss optimality of our algorithm**

**We will ask our algorithm to find optimal path between A and B, in forward manner**

## Data Loading

## Distance Filter
We made small map of several stations

First, we filter on only the stations in a 10km radius around node A. This allows for computation in "reasonable" time on the cluster

**For further analysis, it is just important to note that B to D distance is 400m and we consider this distance as walkable**

In [3]:
def distance_squared(n1,e1, n2, e2):
    '''Calculates the euclidean distance between two points'''
    eucl_dist2 = ((n1-n2)*(n1-n2)+ (e1-e2)*(e1-e2))
    return eucl_dist2

In [4]:
coords_zurich = (0, 0)

koord_station_names = ["A", "B","C","D","E","F","G","H","I","J","L","M"]
KOORDEs = [0, 5000, 1000, 4600, -1000, -2000, 4600, 4600, 0, 10000, 5000, 100]
KOORDNs = [0, 0, -1000, 0, -1000, -2000, 1300, 2300, -1250, 10000, 100, 0]

df_stops = spark.createDataFrame(zip(koord_station_names, KOORDEs, KOORDNs), schema=["station_name", "KOORDE", "KOORDN"])

In [5]:
df_stops = df_stops.withColumn('dist2', distance_squared(coords_zurich[1], coords_zurich[0], df_stops.KOORDN, df_stops.KOORDE))

In [6]:
df_stops = df_stops.filter(df_stops.dist2<=10000**2)

In [7]:
df_stops.show()

+------------+------+------+--------+
|station_name|KOORDE|KOORDN|   dist2|
+------------+------+------+--------+
|           A|     0|     0|       0|
|           B|  5000|     0|25000000|
|           C|  1000| -1000| 2000000|
|           D|  4600|     0|21160000|
|           E| -1000| -1000| 2000000|
|           F| -2000| -2000| 8000000|
|           G|  4600|  1300|22850000|
|           H|  4600|  2300|26450000|
|           I|     0| -1250| 1562500|
|           L|  5000|   100|25010000|
|           M|   100|     0|   10000|
+------------+------+------+--------+



**Constructing isdaten df**

In [8]:
sbb_station_names = ["A", "D", "A", "B", "A", "C", "B", "C", "I", "E", "F", "D", "G", "H", \
                     "D", "A", "B", "A", "C", "A", "I", "C", "B", "F", "E", "H", "G", "D"]
sbb_departure_times = ["17.10.2018 15:00", "17.10.2018 16:11", "17.10.2018 15:00", "17.10.2018 16:01", "17.10.2018 15:00",\
                   "17.10.2018 15:26", "17.10.2018 16:11", "17.10.2018 15:35", "17.10.2018 15:20", "17.10.2018 15:00",\
                       "17.10.2018 15:49", "17.10.2018 16:13", "17.10.2018 16:45", "17.10.2018 16:55",
                      "17.10.2018 20:00", "17.10.2018 21:11", "17.10.2018 20:35", "17.10.2018 20:20", "17.10.2018 20:00",\
                   "17.10.2018 20:26", "17.10.2018 21:11", "17.10.2018 20:35", "17.10.2018 20:20", "17.10.2018 20:00",\
                       "17.10.2018 20:49", "17.10.2018 21:13", "17.10.2018 21:45", "17.10.2018 21:55"]

sbb_arrival_times = ["17.10.2018 14:59", "17.10.2018 16:10", "17.10.2018 14:58", "17.10.2018 16:00", "17.10.2018 14:58",\
                 "17.10.2018 15:25", "17.10.2018 16:10", "17.10.2018 15:30", "Null", "17.10.2018 14:59", \
                     "17.10.2018 15:50", "17.10.2018 16:11", "17.10.2018 16:43", "17.10.2018 16:50", \
                     "17.10.2018 19:59", "17.10.2018 21:10", "17.10.2018 19:58", "17.10.2018 21:00", "17.10.2018 19:58",\
                 "17.10.2018 20:25", "17.10.2018 21:10", "17.10.2018 20:30", "Null", "17.10.2018 19:59", \
                     "17.10.2018 20:50", "17.10.2018 21:11", "17.10.2018 21:43", "17.10.2018 21:50"]

sbb_skipstop = ["false"]*len(sbb_station_names)
sbb_trip_ids = [1,1,2,2,3,3,4,4,4,5,5,6,6,6,7,7,8,8,9,9,10,10,10,11,11,12,12,12]
p_user = 0.75
probs = [0.9, 0.7, 0.8, 0.95, 0.99, 0.8]
sbb_line_type = ["IAD", "IAD", "IAB", "IAB", "IAC", "IAC", "IBCI", "IBCI", "IBCI", "IAD", "IEF","IEF","IDGH","IDGH","IDGH"]*2

swiss_data = spark.createDataFrame(zip(sbb_station_names, sbb_departure_times, sbb_arrival_times, sbb_trip_ids, sbb_line_type, sbb_skipstop)\
                                 , schema=["HALTESTELLEN_NAME", "ABFAHRTSZEIT", "ANKUNFTSZEIT", "FAHRT_BEZEICHNER", "LINIEN_TEXT", "DURCHFAHRT_TF"])
swiss_data.show(50)

+-----------------+----------------+----------------+----------------+-----------+-------------+
|HALTESTELLEN_NAME|    ABFAHRTSZEIT|    ANKUNFTSZEIT|FAHRT_BEZEICHNER|LINIEN_TEXT|DURCHFAHRT_TF|
+-----------------+----------------+----------------+----------------+-----------+-------------+
|                A|17.10.2018 15:00|17.10.2018 14:59|               1|        IAD|        false|
|                D|17.10.2018 16:11|17.10.2018 16:10|               1|        IAD|        false|
|                A|17.10.2018 15:00|17.10.2018 14:58|               2|        IAB|        false|
|                B|17.10.2018 16:01|17.10.2018 16:00|               2|        IAB|        false|
|                A|17.10.2018 15:00|17.10.2018 14:58|               3|        IAC|        false|
|                C|17.10.2018 15:26|17.10.2018 15:25|               3|        IAC|        false|
|                B|17.10.2018 16:11|17.10.2018 16:10|               4|       IBCI|        false|
|                C|17.10.2018 

In [9]:
cross_table = df_stops.join(swiss_data, swiss_data.HALTESTELLEN_NAME == df_stops.station_name, 'inner').persist()
df_stops = cross_table.select('station_name','KOORDE','KOORDN').distinct()

In [10]:
df_stops = cross_table.select('station_name','KOORDE','KOORDN').distinct()

In [11]:
stops_filter = df_stops.select('station_name')
stops_filter.show()

+------------+
|station_name|
+------------+
|           F|
|           E|
|           B|
|           D|
|           C|
|           A|
|           G|
|           I|
|           H|
+------------+



**Note from here that from A to B starting from 14:59, three routes are possible**

**A-B directly, A-D-B due to acceptable walk between D and B, and A-C-B**

**They take 60, 70+walk and 70 min respectively**

### Train Delay

In [12]:
prob = [0.9, 0.7, 0.8, 0.95, 0.95, 0.99, 0.8, 0.8]
line_types = ["IAD", "IAB", "IAC", "IBCI", "IBCI", "IEF", "IDGH", "IDGH"]
koord_station_names = ["A", "A", "A", "I", "C", "E", "D", "G"]#["D", "B", "C", "B","C","F","G","H"]

delays_df = pd.DataFrame(zip(line_types, koord_station_names, prob)\
                                 , columns=["LINIEN_TEXT", "HALTESTELLEN_NAME", "prob"])
delays_df.head(100)

,LINIEN_TEXT,HALTESTELLEN_NAME,prob
0,IAD,A,0.90
1,IAB,A,0.70
2,IAC,A,0.80
3,IBCI,I,0.95
4,IBCI,C,0.95
5,IEF,E,0.99
6,IDGH,D,0.80
7,IDGH,G,0.80


**Note that probabilities A-B directly 0.7, A-D-B due to acceptable walk between D and B 0.9, and A-C-B 0.8*0.95 = 0.76**

## Processing Data
We now want to read data from the sbb files and extract usefull information

#### Find station accessible by walking 

Finds stations in proximity of station of interest. We used 5 minute as an upperbound on the time of walking. Distance is computed as simply the euclidian distance. 

In [13]:
def euclidiandist(x_a,x_b,y_a,y_b):
    # We assumed that average speed of the user is of 5km/h
    avg_speed = 1.388889
    eucl_dist = math.sqrt((x_a-x_b)*(x_a-x_b)+ (y_a-y_b)*(y_a-y_b))
    return eucl_dist/avg_speed

In [14]:
# first, load all the necessary data
import pickle
with open('./data/station_from_index_testing.pkl', 'rb') as handle:
    station_from_index = pickle.load(handle)
with open('./data/index_from_station_testing.pkl', 'rb') as handle:
    index_from_station = pickle.load(handle)
walking_distances = np.load('./data/walking_distances_testing.npy')
walking_distances = np.sqrt(walking_distances)

# definition of the maximum walking time
MAX_WALKING_TIME = 10*60

In [15]:
def get_walking_stations(current_station, current_time, way, isadvance):
    if way == 'forward':
        # get the distances for all stations
        walking_times = walking_distances[index_from_station[current_station],:]
        # get stations in walking distance
        close_stations = np.argwhere(walking_times<MAX_WALKING_TIME)
        # remove station itself
        close_stations = [i for i in close_stations.flatten().tolist()  \
                         if i!=index_from_station[current_station]]
        # get the names of the stations
        station_names = [station_from_index[i] for i in close_stations]
        # get the estimated arrival times for all stations
        arrival_times = (walking_times[close_stations].flatten() + current_time).tolist()
        # get the other information necessary
        line_type = [None]*len(station_names)
        departure_time = [current_time]*len(station_names)


        # return the tuples we want
        walking_stops_tuples = list(zip(line_type, departure_time, \
                                        station_names, arrival_times))
    else:
        if isadvance :
            current_time = current_time - 180
        # get the distances for all stations
        walking_times = walking_distances[index_from_station[current_station],:]
        # get stations in walking distance
        close_stations = np.argwhere(walking_times<MAX_WALKING_TIME)
        # remove station itself
        close_stations = [i for i in close_stations.flatten().tolist()  \
                         if i!=index_from_station[current_station]]
        # get the names of the stations
        station_names = [station_from_index[i] for i in close_stations]
        # get the estimated arrival times for all stations
        departure_time = (current_time - walking_times[close_stations].flatten()).tolist()
        # get the other information necessary
        line_type = [None]*len(station_names)
        arrival_times = [current_time]*len(station_names)
        # return the tuples we want
        walking_stops_tuples = list(zip(line_type, departure_time, \
                                        station_names, arrival_times))
        
    return walking_stops_tuples

In [16]:
get_walking_stations('B', 1000, "forward", False)

[(None, 1000, 'D', 1400.0)]

In [17]:
get_walking_stations('D', 1000, "backward", False)

[(None, 600.0, 'B', 1000)]

In [18]:
def get_all_possible_stations(df, distance_df, stop_name, desired_time, mode,\
                                   allowed_timewindow=1800, isadvance = False):
    """
    Get all stations accessible from a given point by either a SBB trip or by walking
    
    input: 
    
        df :         df with structure as sbb_df
        distance_df: df of a kind didok_df
        stop_name:   string, name of the station we are heading to or from, when mode is forward for a fixed time of arrival, \
                     or anything else for fixed time of departure

        desired_time: time in seconde (unix time)

        allowed_timewindow: maximum time of waiting at a station in seconds
    
    output:
        a tuple list of all accesible in the format: (type of train, departure time at current station,
                                                       name of the next station, time of arrival at next station)
        NB : if the trip is by walking the type of train is None

    """
    df = df\
        .withColumn('ArrivalTimeScheduled', f.unix_timestamp(f.col("ArrivalTimeScheduled"), "dd.MM.yyyy HH:mm"))\
        .withColumn('DepartureTimeScheduled', f.unix_timestamp(f.col("DepartureTimeScheduled"), "dd.MM.yyyy HH:mm"))
    if(mode == 'forward') :
        in_station = df.filter(df['SkipStation'] == 'false').filter(df['DepartureTimeScheduled'] > desired_time).filter(df['DepartureTimeScheduled'] < desired_time + allowed_timewindow).filter(df['station_name'] == stop_name).select('TripId','DepartureTimeScheduled')
        next_station = df.filter(df['SkipStation'] == 'false').select('StopName','TripId','ArrivalTimeScheduled','TransportType').na.fill("Unknown")
        both = next_station.filter(next_station['StopName'] != stop_name).filter(next_station['ArrivalTimeScheduled'] < desired_time + 28800).join(in_station,on = 'TripId')
        res = both.filter(both['ArrivalTimeScheduled'] > both['DepartureTimeScheduled'] ).select('TransportType','DepartureTimeScheduled','StopName','ArrivalTimeScheduled')
        
        tuples_train = res.rdd.map(lambda row:(row[0], row[1], row[2], row[3])).collect()
        
    else :
        # We divide the allowed time window by two in the case that the arrival time is fixed, assuming that tolerence for waiting in arrival point is lower that at departure 
        in_station = df.filter(df['SkipStation'] == 'false').filter(df['ArrivalTimeScheduled'] < desired_time).filter(df['ArrivalTimeScheduled'] > desired_time - allowed_timewindow/2).filter(df['station_name'] == stop_name).select('TripId','ArrivalTimeScheduled')

        next_station = df.filter(df['SkipStation'] == 'false').select('StopName','TripId','DepartureTimeScheduled','TransportType').na.fill("Unknown")

        both = next_station.filter(next_station['StopName'] != stop_name).filter(next_station['DepartureTimeScheduled'] > desired_time - 28800).join(in_station,on = 'TripId')
        res = both.filter(both['ArrivalTimeScheduled'] > both['DepartureTimeScheduled'] ).select('TransportType','DepartureTimeScheduled','StopName','ArrivalTimeScheduled')
        tuples_train = res.rdd.map(lambda row:(row[0], row[1], row[2], row[3])).collect()
    tuples_with_walking = get_walking_stations(stop_name, desired_time, mode, isadvance)
    all_tuples = tuples_train + tuples_with_walking
    
    
    return all_tuples

In [19]:
def nice_time(t):
    """Useful for seeing time for debugging"""
    return datetime.fromtimestamp(abs(t)).strftime("%d.%m.%Y %H:%M")

Let's translate dataframe

In [20]:
istdaten = swiss_data.join(stops_filter, swiss_data.HALTESTELLEN_NAME == stops_filter.station_name, 'inner')
istdaten.show(50)

+-----------------+----------------+----------------+----------------+-----------+-------------+------------+
|HALTESTELLEN_NAME|    ABFAHRTSZEIT|    ANKUNFTSZEIT|FAHRT_BEZEICHNER|LINIEN_TEXT|DURCHFAHRT_TF|station_name|
+-----------------+----------------+----------------+----------------+-----------+-------------+------------+
|                A|17.10.2018 15:00|17.10.2018 14:59|               1|        IAD|        false|           A|
|                D|17.10.2018 16:11|17.10.2018 16:10|               1|        IAD|        false|           D|
|                A|17.10.2018 15:00|17.10.2018 14:58|               2|        IAB|        false|           A|
|                B|17.10.2018 16:01|17.10.2018 16:00|               2|        IAB|        false|           B|
|                A|17.10.2018 15:00|17.10.2018 14:58|               3|        IAC|        false|           A|
|                C|17.10.2018 15:26|17.10.2018 15:25|               3|        IAC|        false|           C|
|         

In [21]:
traduction = ' StopName string, DepartureTimeScheduled string, ArrivalTimeScheduled string, TripId string, TransportType string, SkipStation boolean'
##, LineType string,
traduction = list(map(lambda x: x.split()[0],traduction.split(',')))

for german, english in zip(istdaten.columns, traduction):
    istdaten = istdaten.withColumnRenamed(german, english)
istdaten.show(50)

+--------+----------------------+--------------------+------+-------------+-----------+------------+
|StopName|DepartureTimeScheduled|ArrivalTimeScheduled|TripId|TransportType|SkipStation|station_name|
+--------+----------------------+--------------------+------+-------------+-----------+------------+
|       A|      17.10.2018 15:00|    17.10.2018 14:59|     1|          IAD|      false|           A|
|       D|      17.10.2018 16:11|    17.10.2018 16:10|     1|          IAD|      false|           D|
|       A|      17.10.2018 15:00|    17.10.2018 14:58|     2|          IAB|      false|           A|
|       B|      17.10.2018 16:01|    17.10.2018 16:00|     2|          IAB|      false|           B|
|       A|      17.10.2018 15:00|    17.10.2018 14:58|     3|          IAC|      false|           A|
|       C|      17.10.2018 15:26|    17.10.2018 15:25|     3|          IAC|      false|           C|
|       B|      17.10.2018 16:11|    17.10.2018 16:10|     4|         IBCI|      false|    

In [23]:
def parse_tuple(all_stations):
    for row in all_stations:
        print(row[0], nice_time(row[1]), row[2], nice_time(row[3]))
    return

In [24]:
import time

stop_name = "B"
mode = "backward"
desired_time = time.mktime(datetime.strptime("17.10.2018 16:15", "%d.%m.%Y %H:%M").timetuple())
window = 1800
print("Desired time at B : ", desired_time, nice_time(desired_time))
all_stations = get_all_possible_stations(istdaten, df_stops, stop_name, desired_time, mode, window)
all_stations
parse_tuple(all_stations)

Desired time at B :  1539785700.0 17.10.2018 16:15
IBCI 17.10.2018 15:35 C 17.10.2018 16:10
IBCI 17.10.2018 15:20 I 17.10.2018 16:10
None 17.10.2018 16:08 D 17.10.2018 16:15


In [25]:
import time

stop_name = "A"
mode = "forward"
desired_time = time.mktime(datetime.strptime("17.10.2018 14:59", "%d.%m.%Y %H:%M").timetuple())
window = 1800
print("Desired time at A : ", desired_time, nice_time(desired_time))
all_stations = get_all_possible_stations(istdaten, df_stops, stop_name, desired_time, mode, window)
all_stations
parse_tuple(all_stations)

Desired time at A :  1539781140.0 17.10.2018 14:59
IAD 17.10.2018 15:00 D 17.10.2018 16:10
IAC 17.10.2018 15:00 C 17.10.2018 15:25
IAB 17.10.2018 15:00 B 17.10.2018 16:00


## Dealing with probability

Here we deal with the fact that the time of arrival is not determinitic but a random variable. If we need to define it by a single value we decided to use the expectency, which is used to calculate the estimated time of arrival. The delay probability is used to find the probability for a train to arrive early enough that the user can safly catch the next train. 

In [26]:
def get_expected_delay(station_name, line_id):
    return 0 # average expected delay over dataset is 0, congrats SBB ;)

In [27]:
import warnings
def get_delay_probability(station_name, line_id, delay):
    row = delays_df.loc[(delays_df.LINIEN_TEXT == line_id) & (delays_df.HALTESTELLEN_NAME==station_name)]
    if row.shape[0]>0:
        return row["prob"].values[0]
    else:
        return delays_df["prob"].mean()

In [28]:
import pickle
with open('./data/stations_dict_testing', 'rb') as handle:
    stations_dict = pickle.load(handle)
min_travel_times = np.load('./data/min_travel_times_testing.npy')

def get_heuristic(city_a,city_b):
    '''
    Calculates the heuristic of time in minutes between two city taking a train
    '''
    # try to find connection in our adjacency matrix calculated in Heuristic.ipynb
    if city_a in stations_dict and city_b in stations_dict:
        i = stations_dict[city_a]
        j = stations_dict[city_b]
        estimated_time = min_travel_times[i,j]
        if estimated_time!=1440:
            # estimated_time=1440: no possible connection was found when building the Heuristic adjacency matrix
            return float(estimated_time)
    
    avg_speed = 1600
    try :
        df_a = df_stops.select('KOORDE','KOORDN').where(df_stops['station_name'] ==city_a).collect()
        x_a = float(df_a[0]['KOORDE'])
        y_a = float(df_a[0]['KOORDN'])
    except IndexError :
        print('Name ',city_a,'is invalide')
        return 0
    try :
        df_b = df_stops.select('KOORDE','KOORDN').where(df_stops['station_name'] ==city_b).collect()
        x_b = float(df_b[0]['KOORDE'])
        y_b = float(df_b[0]['KOORDN'])
    except IndexError :
        print('Name ',city_b,'is invalide')
        return 0
    eucl_dist = math.sqrt((x_a-x_b)*(x_a-x_b)+ (y_a-y_b)*(y_a-y_b))
    return eucl_dist/avg_speed

In [29]:
class Step :
    def __init__(self,departure_station,arrival_station, depature_time,arrival_time, train):
        self.departure_station = departure_station
        self.arrival_station = arrival_station
        self.depature_time = depature_time
        self.arrival_time = arrival_time
        self.train = train
    def _get(self) :
        #print('From : ',self.departure_station,',to : ', self.arrival_station, ',departing at : ', self.depature_time,
        #     'with train', self.train, '\n')
        return {'departure' : self.departure_station, 'destination' : self.arrival_station, \
                'time of departure' : datetime.fromtimestamp(abs(self.depature_time)).strftime('%Y-%m-%d %H:%M:%S'),\
                'time of arrival' : datetime.fromtimestamp(abs(self.arrival_time)).strftime('%Y-%m-%d %H:%M:%S'),\
                'type' : self.train}

# Station keep each station and it's associated heuristic 
class Station :
    def __init__(self,name, heuristic):
        self.name = name
        self.heuristic = heuristic
    def _print(self) :
        print('Station ',self.name,'with heursitic ', self.heuristic)

# State describe a station visited at a certain time from a certain path, with a certain probability 
class State :
    def __init__(self,station, eta, ps, previous_steps,last_train,pred_arrival_time):
        # Name of the station at that state
        self.station = station
        
        # Estimated time of arrival at that state. (in backward mode this is estimated time of departure)
        self.eta = eta
        
        # Probability to be at that state (given by probability to be at last state * probability to catch connection
        #to previous state)
        self.ps = ps
        
        # List of the step we took to get to that state
        self.previous_steps = previous_steps
        
        # The linien_text of the last taken train
        self.last_train = last_train
        
        # The predicted arrival time to that station (in backward mode this is predicted depature time of last transition)
        self.pred_arrival_time = pred_arrival_time
    def get_steps(self) :
        steps_list = []
        if (len(self.previous_steps) != 0):
            for step in self.previous_steps :
                steps_list.append(step._get())
        else : print('Leave from home')
        return steps_list
    def __lt__ (self,other) :
        return self.eta < other.eta
    def __hash__ (self) :
        return hash(self.eta)  
    def _print(self) :
        print('At',self.station.name,' the estimated time of arrival is ',self.eta\
              ,' with a probability of being here of ',self.ps, '\n')
        
class End(Exception):
    """We found the right station"""
    pass
class Empty_queue(Exception):
    """Parameter where too restrictive, no path found and priority queue is empty"""
    pass

## Implementating the shortest path search 

In [30]:
def createstate(state,next_,destination,Pthresh,way):
    """Create a state resulting of a trip between station a and b :
    Input : 
        - current state s at station a
        - tuple containing the linien_text of the train t that goes from a to b, predicted
        departure time at station a, predicted arrival time at station b and name of station b
        - Final destination we are looking for and minimum probability we need. 
    Output :
        - New State resulting from trip from a to b """
    linien_text = next_[0]
    station_name = next_[2]
    pred_departure = next_[1] 
    pred_arrival = next_[3]
    # first we estimate the heuristic for this new station
    heuristic = get_heuristic(station_name,destination)
    station = Station(station_name,heuristic)
    
    
    if(linien_text != None) :
        # If we go to the next station by train
        # get expected probability 
        Expected_delay = get_expected_delay(linien_text,station_name)
        if (way == 'forward') :
            # Calculate the new ETA
            ETA = Expected_delay*60 + heuristic*60 + pred_arrival
             # Calculates the probability that the last train arrives early enough to catch the next one (3min changing time)
            delay = pred_departure- state.pred_arrival_time - 3*60
            Pboard = get_delay_probability(state.station.name,linien_text,delay/60)
            #Calculate the probability of the state which is the probability of the last state * probability of having made
            # it to the next trip on time
            Ps = state.ps * Pboard

            # Update the step list 
            new_step = Step(state.station.name,station_name,pred_departure,pred_arrival,linien_text)
        
            Previous = state.previous_steps + [new_step]
            return State(station,ETA,Ps,Previous,linien_text,pred_arrival)
        else :
            # Since the train departure time are assumed to be deterministic their is no delay of departure also eta is really estimated departure time
            # in this way of calculation and must be negatuve as the priority list is selecting on smaller element
            ETD = heuristic*60 - pred_departure
             # Calculates the probability that the last train arrives early enough to catch the next one (3min changing time)
            # NB here state.pred_arrival_time is actually the depature time of the next connection
            delay = state.pred_arrival_time - pred_arrival
            Pboard = get_delay_probability(state.station.name,linien_text,delay/60)
            #Calculate the probability of the state which is the probability of the last state * probability of having made
            # it to the next trip on time
            Ps = state.ps * Pboard

            # Update the step list 
            new_step = Step(station_name,state.station.name,pred_departure,pred_arrival,linien_text) 
            Previous = [new_step] + state.previous_steps 
            return State(station,ETD,Ps,Previous,linien_text,pred_departure)
        
        
       
    else : 
        # if we walked from the last station  
        walking_time = pred_arrival - pred_departure 
        if (way == 'forward') :
            ETA = state.eta - state.station.heuristic*60 + walking_time + heuristic*60
            # if we walk we can leave when we want
            Ps = state.ps
            # Update the step list 
            new_step = Step(state.station.name,station_name,pred_departure,pred_arrival,'Walk')
            Previous = state.previous_steps + [new_step]
            return State(station,ETA,Ps,Previous,linien_text,pred_arrival)
        else :
            ETD = state.eta - state.station.heuristic*60 +  heuristic*60 + walking_time
            # we assume that we must arrive 3 minute in advance or more to catch the train
            Ps = state.ps
            # Update the step list 
            new_step = Step(station_name,state.station.name,pred_departure,pred_arrival,'Walk')
            Previous = [new_step] + state.previous_steps
            return State(station,ETD,Ps,Previous,linien_text,pred_departure)
    
        
    
    

In [31]:
def firststep(From, To, t, Pthresh,way):
    t = float(time.mktime(datetime.strptime(t, "%d.%m.%Y %H:%M").timetuple()))
    start_heuristic = get_heuristic(From,To)
    pqueue = PriorityQueue()
    pqueue.put((float('inf'),"empty_queue"))
    if (way == 'forward') :
        current_station = Station(From,start_heuristic)
        start_state = State(current_station,start_heuristic*60+t,1.0,[],"starting",t)
        pqueue.put((start_state.eta,start_state))
    else :
        current_station = Station(To,start_heuristic)
        start_state = State(current_station,start_heuristic*60-t,1.0,[],"starting",t)
        pqueue.put((start_state.eta,start_state))
    visited = visited_nodes()
    visited.update(start_state,way)
    return visited,pqueue

In [32]:
def new_step(priortity_list,Pthresh,destination,visited,way,reduced_df) :
    """For the first state in the priority_list find all possible next state and insert them into the priortity list 
    if their probability is highy<Ser than Pthresh"""
    current_state = priortity_list.get()[1]
    if (type(current_state) == str) :
        raise Empty_queue("error")
    
    #print(current_state.station.name,nice_time(current_state.eta),current_state.ps,current_state.last_train,len(current_state.previous_steps))
    if (current_state.station.name == destination):
        raise End(8,current_state)
    # Get all stations to which we can go from current state
    # Let's assume that getpossibletrip returns a list of tuple like 
    # (linien_text,next_station_name,predicted_arrival_time_current_station,predicted_departure_time, predicted_arrival_time) 
    # for each possible new trip. Predicted_departure_time is the predicted departure time at current state
    # predicted_arrival_time the predicted arrival time in the reachable station
    # If we are walking train_id is None, departure is the arrival time of last trip
    # and arrival is departure time + estimation for walking time. 
    if (way == 'forward'):
        
        possible_next_stops = get_all_possible_stations(reduced_df,df_stops,current_state.station.name,current_state.pred_arrival_time,way)
        
    else :
        if(current_state.last_train != "starting"):
            # if we have a connections to catch and we are walking we should arrive 3 minutes in advance 
            possible_next_stops = get_all_possible_stations(reduced_df,df_stops,current_state.station.name,current_state.pred_arrival_time,way,isadvance = True)
        else :
            # if we are walking to destination we can arrive at desired time  
            possible_next_stops = get_all_possible_stations(reduced_df,df_stops,current_state.station.name,current_state.pred_arrival_time,way)
        
    # For each next station, lets create the next state and add it to the Priority Queue
    for next_ in possible_next_stops :
        #print(next_[2],'leaving at ',nice_time(next_[1]), 'arriving at',nice_time(next_[3]),'using',next_[0])
        if (current_state.last_train == None and next_[0] == None) :
            # Here we want to avoid to do twice a trip by walking
            continue
        n_state = createstate(current_state,next_,destination,Pthresh,way)
        if n_state.ps > Pthresh and visited.update(n_state,way) :
            
            priortity_list.put((n_state.eta,n_state))
            
    return priortity_list, visited


## Visited nodes
We want to model the set of visited nodes in a way that there are duplicates in the visited nodes only if the node has a higher probability of arriving there in a slower time. In all other cases only the fastest achieved node is saved in the set.

In [33]:
 def create_tuple_from_node(node):
        return (node.station.name, node.eta, node.ps)

In [34]:
#visited_nodes as a set of tuples, containing: (station, visit-time, certainity)
class visited_nodes:
    def __init__(self, initial_set=set([])):
        self.nodes = initial_set
    def update(self, new_node,way):
        old_nodes = self.previous_visits(new_node)
        new_node_tup = create_tuple_from_node(new_node)
        
        if len(old_nodes) == 0: #case when the node has not been explored yet.
            self.add_to_visited(new_node_tup)
            return True
         
        else: #case when the node has been explored and multiple optimal values are in the set
            added = False
            for n in old_nodes: #iterate through nodes and compare to new node
                if (way == 'forward') :
                    if self.compare_nodes_2_forward(n, new_node_tup): #node is added if better/equal than any node and replaces worse nodes
                        added = True
                else :
                    if self.compare_nodes_2_backward(n, new_node_tup): #node is added if better/equal than any node and replaces worse nodes
                        added = True
            self.compare_in_set(new_node,way)
            return added
            
    '''previous_visits takes a node and compares it to the already existing set of nodes that have been visited.
        If a node has been visited before, the function hands out the previous visits in a list'''
    def previous_visits(self, new_node):
        return list(filter(lambda x: x[0] == new_node.station.name, self.nodes))
    
    '''add_to_visited takes a new node and adds it to the set of visited nodes'''
    def add_to_visited(self, new_node_tup):
        self.nodes.add(new_node_tup)
        return None
    
    '''replace_old is used when a better node (higher probability and faster) appears. It takes a new node and adds
        it to the set of visited nodes, while deleting the old one'''
    def replace_old(self, new_node, old_node):
        self.add_to_visited(new_node)
        if old_node in self.nodes:
            self.nodes.remove(old_node)
        return None

    
    '''takes two nodes and compares them according to the arrival time and probability. If a node is in some way
        better than the existing it is stored. The function returns a boolean which indicates whether the value 
        has been stored (important to know for updating priority queue)'''
    def compare_nodes_2_forward(self, node1, node2):
    # Compare node by estimated time of arrival in which a lower eta is a fastest way
        if node1[1] > node2[1] and node1[2] < node2[2]: #the node2 is faster and has a higher proba
            self.replace_old(node2, node1) #we replace the slow and improbable node
            return True

        elif node1[1] > node2[1] and node1[2] > node2[2]: #the node2 is only faser, but less certain
            self.add_to_visited(node2) # add to optimal set
            return True

        elif node1[1] < node2[1] and node1[2] < node2[2]: #the node2 is slower but more certain
            self.add_to_visited(node2) #add it to optimal set
            return True

        elif node1[1] > node2[1] and node1[2] == node2[2]:
            self.replace_old(node2, node1)
            return True

        elif node1[1] == node2[1] and node1[2] < node2[2]:
            self.replace_old(node2, node1)
            return True

        else:
            return False #the node2 is neither more certain nor faster, we drop it.
    def compare_nodes_2_backward(self, node1, node2):
        # Compare node by estimated time of departure in which a bigger etd is a fastest way
        if node1[1] > node2[1] and node1[2] < node2[2]: #the node2 is faster and has a higher proba
            self.replace_old(node2, node1) #we replace the slow and improbable node
            return True

        elif node1[1] < node2[1] and node1[2] > node2[2]: #the node2 is only faser, but less certain
            self.add_to_visited(node2) # add to optimal set
            return True

        elif node1[1] > node2[1] and node1[2] < node2[2]: #the node2 is slower but more certain
            self.add_to_visited(node2) #add it to optimal set
            return True

        elif node1[1] < node2[1] and node1[2] == node2[2]:
            self.replace_old(node2, node1)
            return True

        elif node1[1] == node2[1] and node1[2] < node2[2]:
            self.replace_old(node2, node1)
            return True

        else:
            return False #the node2 is neither more certain nor faster, we drop it.
            
    def compare_in_set(self, new_node,way):
        old_nodes = self.previous_visits(new_node)
        for n in old_nodes:
            for m in old_nodes:
                if (way == 'forward'):
                    self.compare_nodes_2_forward(n, m)
                else :
                    self.compare_nodes_2_backward(n, m)


## Main function

In [35]:
def main_toy(From,To,At,P,way):
    steps = None
    # initate the queue 
    visited,queue = firststep(From,To,At,P,way)
    # load the Data for the day of the trip as well as one day before/after to account for midnight trip.
    t = datetime.strptime(At, "%d.%m.%Y %H:%M").timetuple()    
    # Running the query until the final station is found 
    
    while True:
        try:
            if (way =='forward'):
                queue,visited = new_step(queue,P,To,visited,way,istdaten)
            else :
                queue,visited = new_step(queue,P,From,visited,way,istdaten)
        except End as end: 
            # We found the soltution
            final_state = end.args[1]
            steps = pd.DataFrame(final_state.get_steps())
            break
        except Empty_queue:
            print('No way was found with this probability threshold, try again with a lower threshold or stay home')
            break
    return steps

## A few run to try 

In [36]:
def nice_time(t):
    """Useful for seeing time for debugging"""
    return datetime.fromtimestamp(abs(t)).strftime("%d.%m.%Y %H:%M")

In [37]:
start = time.time()

steps = main_toy('A','B','17.10.2018 14:59',0.7,'forward')

end = time.time()
print('took {}mins'.format((end - start)/60))
steps

took 0.12388428052266438mins


,departure,destination,time of arrival,time of departure,type
0,A,C,2018-10-17 15:25:00,2018-10-17 15:00:00,IAC
1,C,B,2018-10-17 16:10:00,2018-10-17 15:35:00,IBCI


**We observe that our algorithm did not picked shotest route AB neither longer one A-D-B, it picked the one with minimal time for a given probability**